In [2]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense
from tensorflow.keras.utils import to_categorical
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv('train.csv')

df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['text'])
vocab_size = len(tokenizer.word_index) + 1

In [7]:
sequences = tokenizer.texts_to_sequences(df['text'])
max_len = max(len(seq) for seq in sequences)
X = pad_sequences(sequences,max_len,padding='post')

In [9]:
embedding_dim = 50
embedding_matrix = np.zeros((vocab_size,embedding_dim))

In [12]:
glove_file_path = "C:\\Users\\Rushil Misra\\Downloads\\glove.6B\\glove.6B.50d.txt"
with open(glove_file_path, 'r', encoding= 'utf-8') as file:
    for line in file:
        values = line.split()
        word = values[0]
        if word in tokenizer.word_index:
            index = tokenizer.word_index[word]
            embedding_matrix[index] = np.array(values[1:], dtype = 'float32')

In [35]:
model = Sequential()
model.add(Embedding(vocab_size,embedding_dim,weights = [embedding_matrix], input_length = max_len, trainable = False))
model.add(Flatten())
model.add(Dense(64,activation="relu"))
model.add(Dense(1,activation = "sigmoid"))

In [36]:
model.compile(optimizer='adam',loss = 'binary_crossentropy', metrics = ['accuracy'])

labels = df['target']
model.fit(X,labels,epochs=10,batch_size=32,validation_split = 0.2)


Epoch 1/10
191/191 [==============================] - 2s 8ms/step - loss: 0.5114 - accuracy: 0.7547 - val_loss: 0.4954 - val_accuracy: 0.7525
Epoch 2/10
191/191 [==============================] - 1s 5ms/step - loss: 0.3968 - accuracy: 0.8300 - val_loss: 0.4846 - val_accuracy: 0.7715
Epoch 3/10
191/191 [==============================] - 1s 3ms/step - loss: 0.3331 - accuracy: 0.8639 - val_loss: 0.5274 - val_accuracy: 0.7636
Epoch 4/10
191/191 [==============================] - 1s 3ms/step - loss: 0.2734 - accuracy: 0.8947 - val_loss: 0.5181 - val_accuracy: 0.7571
Epoch 5/10
191/191 [==============================] - 1s 3ms/step - loss: 0.2174 - accuracy: 0.9240 - val_loss: 0.5755 - val_accuracy: 0.7544
Epoch 6/10
191/191 [==============================] - 1s 3ms/step - loss: 0.1757 - accuracy: 0.9427 - val_loss: 0.6058 - val_accuracy: 0.7643
Epoch 7/10
191/191 [==============================] - 1s 3ms/step - loss: 0.1428 - accuracy: 0.9588 - val_loss: 0.6412 - val_accuracy: 0.7564
Epoch 

In [37]:
test = pd.read_csv('test.csv')
test_sequences = tokenizer.texts_to_sequences(test['text'])
X_test = pad_sequences(test_sequences, maxlen=max_len, padding='post')

prediction = model.predict(X_test)
binary_predictions = (prediction > 0.5).astype(int)


102/102 [==============================] - 0s 2ms/step


In [23]:
print(len(prediction))

3263


In [25]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3263 entries, 0 to 3262
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        3263 non-null   int64 
 1   keyword   3237 non-null   object
 2   location  2158 non-null   object
 3   text      3263 non-null   object
dtypes: int64(1), object(3)
memory usage: 102.1+ KB


In [38]:

predicted_data = pd.DataFrame()
predicted_data['id'] = test['id']
predicted_data['target'] = binary_predictions

predicted_data.to_csv("DL_predicted_data.csv",index=False)



In [20]:
predicted_data.head()

TypeError: cannot use a string pattern on a bytes-like object

TypeError: cannot use a string pattern on a bytes-like object